In [6]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("SCADA-Forecasting")
    .config("spark.driver.memory", "8g")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.shuffle.partitions", "50")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .getOrCreate()
)

In [7]:
df = spark.read.parquet(r"scada_prepro.parquet")

In [8]:
import os
import sys
from pathlib import Path

notebook_dir = Path(os.getcwd())
project_root = notebook_dir / "../../.."
project_root = project_root.resolve()

sdk_path = project_root / "src" / "sdk" / "python"
sdk_path = sdk_path.resolve()

sys.path.insert(0, str(sdk_path))

from rtdip_sdk.pipelines.forecasting.spark.autogluon_timeseries import AutoGluonTimeSeries

INFO:py4j.clientserver:Error while sending or receiving.
Traceback (most recent call last):
  File "/opt/conda/envs/rtdip-sdk/lib/python3.12/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
INFO:py4j.clientserver:Closing down clientserver connection
INFO:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/rtdip-sdk/lib/python3.12/site-packages/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/rtdip-sdk/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/con

<h2>Warning</h2>
There is a chance that the cell below fails and kills your Spark Session. That's because of the "df.toPandas()" within the split_data() function. toPandas() works totally fine for smaller datasets, but it likes to cause an out of memory error for bigger datasets 

In [4]:
ag_model = AutoGluonTimeSeries()
train_df, test_df = ag_model.split_data(df)

25/12/01 12:24:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):

  if LooseVersion(pyarrow.__version__) < LooseVersion(minimum_pyarrow_version):

  if is_categorical_dtype(series.dtype):



In [5]:
ag_model.train(train_df)

25/12/01 12:25:11 WARN TaskSetManager: Stage 2 contains a task of very large size (2904 KiB). The maximum recommended task size is 1000 KiB.
  if not is_datetime64tz_dtype(pser.dtype):

  if is_datetime64tz_dtype(s.dtype):

  if isinstance(node, ast.Num):  # <number>

  return node.n

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to '/workspaces/amos2025ws03-rtdip-timeseries-forecasting/amos_team_resources/scada/training/AutogluonModels/ag-20251201_122513'
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025
CPU Count:          12
GPU Count:          0
Memory Avail:       9.00 GB / 15.58 GB (57.8%)
Disk Space Avail:   141.91 GB / 930.63 GB (15.2%)
Setting presets to: medium_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'freq': 'h',
 'hyperparamet

In [9]:
ag_model.evaluate(test_df)

  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):

  if LooseVersion(pyarrow.__version__) < LooseVersion(minimum_pyarrow_version):

25/12/01 12:30:03 WARN TaskSetManager: Stage 4 contains a task of very large size (3628 KiB). The maximum recommended task size is 1000 KiB.
  if not is_datetime64tz_dtype(pser.dtype):

  if is_datetime64tz_dtype(s.dtype):

Exception ignored in: <function ResourceTracker.__del__ at 0x75694b1b9c60>
Traceback (most recent call last):
  File "/opt/conda/envs/rtdip-sdk/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/opt/conda/envs/rtdip-sdk/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/opt/conda/envs/rtdip-sdk/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: Exception ignored in: <function ResourceTracker.__del__ at 0x7c0729691c60>
<function ResourceTracker.__del__ a

{'MAE': -77.4486881299248,
 'RMSE': -104.48807855619467,
 'MAPE': -25.78858788352611,
 'MASE': -0.14624511436852122,
 'SMAPE': -1.2321969801358172}

In [10]:
ag_model.get_leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-231.473057,1.766015,0.207669,9
1,Chronos[bolt_small],-239.216585,1.558170,3.045580,7
2,TemporalFusionTransformer,-307.746506,0.207845,262.679121,8
3,DirectTabular,-379.100685,0.133357,2.231830,4
4,RecursiveTabular,-431.559097,0.094970,1.381762,3
5,Naive,-508.930175,2.188388,0.048687,1
6,ETS,-508.930979,1.300879,0.056821,5
7,Theta,-544.990849,0.571940,0.059559,6
8,SeasonalNaive,-598.665711,0.019139,0.059695,2
